In [1]:
import pandas as pd
import os

# =============================
# PATH CONFIG
# =============================
PIPELINE_FILES = {
    "Raw Zone": {
        "path": r"C:\Users\USER\Desktop\BI-DM Project\data\Raw_zone\raw_data.csv",
        "required_columns": []
    },
    "Staging Zone": {
        "path": r"C:\Users\USER\Desktop\BI-DM Project\data\Staging_zone\staging_data.csv",
        "required_columns": ["country_name", "year", "value"]
    },
    "Cleansing Zone": {
        "path": r"C:\Users\USER\Desktop\BI-DM Project\data\Cleansing_zone\clean_data.csv",
        "required_columns": ["country", "year", "co2"]
    },
    "Presentation Zone": {
        "path": r"C:\Users\USER\Desktop\BI-DM Project\data\presentation_zone\presentation.csv",
        "required_columns": ["country", "year", "total_co2"]
    },
    "Prediction Zone": {
        "path": r"C:\Users\USER\Desktop\BI-DM Project\data\Prediction_zone\test_data.csv",
        "required_columns": ["country", "year", "co2"]
    }
}

# =============================
# TEST FUNCTION
# =============================
def test_pipeline_zone(zone_name, config):
    print(f"\n🔍 Testing {zone_name}")
    print("-" * 40)

    path = config["path"]
    required_columns = config["required_columns"]

    # 1️⃣ Check file exists
    if not os.path.exists(path):
        print("❌ FAIL: File not found")
        return False

    print("✅ File exists")

    # 2️⃣ Read file
    try:
        df = pd.read_csv(path)
        print("✅ File loaded successfully")
    except Exception as e:
        print("❌ FAIL: Cannot read file:", e)
        return False

    # 3️⃣ Check rows
    if len(df) == 0:
        print("❌ FAIL: No data rows")
        return False
    else:
        print(f"✅ Rows count: {len(df)}")

    # 4️⃣ Check required columns
    missing_cols = [col for col in required_columns if col not in df.columns]
    if missing_cols:
        print("❌ FAIL: Missing columns:", missing_cols)
        return False
    else:
        if required_columns:
            print("✅ Required columns OK")

    print("🎉 RESULT: PASS")
    return True


# =============================
# RUN ALL TESTS
# =============================
print("\n🚀 DATA PIPELINE TEST REPORT")
print("=" * 50)

passed = 0
failed = 0

for zone, config in PIPELINE_FILES.items():
    result = test_pipeline_zone(zone, config)
    if result:
        passed += 1
    else:
        failed += 1

# =============================
# SUMMARY
# =============================
print("\n📊 TEST SUMMARY")
print("=" * 50)
print(f"✅ Passed Zones : {passed}")
print(f"❌ Failed Zones : {failed}")
print(f"📦 Total Zones  : {passed + failed}")

if failed == 0:
    print("\n🎉 ALL DATA PIPELINE TESTS PASSED")
else:
    print("\n⚠️ SOME PIPELINE TESTS FAILED — CHECK LOG ABOVE")



🚀 DATA PIPELINE TEST REPORT

🔍 Testing Raw Zone
----------------------------------------
✅ File exists
✅ File loaded successfully
✅ Rows count: 13953
🎉 RESULT: PASS

🔍 Testing Staging Zone
----------------------------------------
✅ File exists
✅ File loaded successfully
✅ Rows count: 13953
✅ Required columns OK
🎉 RESULT: PASS

🔍 Testing Cleansing Zone
----------------------------------------
✅ File exists
✅ File loaded successfully
✅ Rows count: 13953
✅ Required columns OK
🎉 RESULT: PASS

🔍 Testing Presentation Zone
----------------------------------------
✅ File exists
✅ File loaded successfully
✅ Rows count: 13953
✅ Required columns OK
🎉 RESULT: PASS

🔍 Testing Prediction Zone
----------------------------------------
✅ File exists
✅ File loaded successfully
✅ Rows count: 2804
✅ Required columns OK
🎉 RESULT: PASS

📊 TEST SUMMARY
✅ Passed Zones : 5
❌ Failed Zones : 0
📦 Total Zones  : 5

🎉 ALL DATA PIPELINE TESTS PASSED


In [9]:
# prediction zone (multi-country)
import pandas as pd
from sklearn.model_selection import train_test_split

# -----------------------------
# 1. Load clean data
# -----------------------------
df = pd.read_csv(
    r"C:\Users\USER\Desktop\BI-DM Project\data\Cleansing_zone\clean_data.csv"
)

# ใช้เฉพาะ field ที่จำเป็น
df = df[["country", "year", "co2"]]

train_list = []
test_list = []

# -----------------------------
# 2. Train / Test split per country
# -----------------------------
for country, group in df.groupby("country"):
    # ต้องมีข้อมูลมากพอ
    if len(group) < 5:
        continue

    X = group[["year"]]
    y = group["co2"]

    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.2,
        random_state=42
    )

    train_df = X_train.assign(
        country=country,
        co2=y_train
    )

    test_df = X_test.assign(
        country=country,
        co2=y_test
    )

    train_list.append(train_df)
    test_list.append(test_df)

# -----------------------------
# 3. Combine all countries
# -----------------------------
train_data = pd.concat(train_list).reset_index(drop=True)
test_data = pd.concat(test_list).reset_index(drop=True)

# -----------------------------
# 4. Save to Prediction Zone
# -----------------------------
train_data.to_csv(
    r"C:\Users\USER\Desktop\BI-DM Project\data\Prediction_zone\train_data.csv",
    index=False
)

test_data.to_csv(
    r"C:\Users\USER\Desktop\BI-DM Project\data\Prediction_zone\test_data.csv",
    index=False
)

print("✅ Prediction data prepared for multiple countries")
print(f"📊 Train rows: {len(train_data)}")
print(f"🧪 Test rows: {len(test_data)}")


✅ Prediction data prepared for multiple countries
📊 Train rows: 11146
🧪 Test rows: 2804


In [7]:
#presentation zone
import pandas as pd

df = pd.read_csv(
    r"C:\Users\USER\Desktop\BI-DM Project\data\Cleansing_zone\clean_data.csv"
)

# สรุป CO₂ ต่อประเทศต่อปี
presentation_df = (
    df.groupby(["country", "year"])["co2"]
    .sum()
    .reset_index()
)

presentation_df.to_csv(
    r"C:\Users\USER\Desktop\BI-DM Project\data\presentation_zone\presentation.csv",
    index=False
)

print(presentation_df.head())



       country  year      co2
0  Afghanistan  1960  414.371
1  Afghanistan  1961  491.378
2  Afghanistan  1962  689.396
3  Afghanistan  1963  707.731
4  Afghanistan  1964  839.743


In [6]:
#cleansing zone
import pandas as pd

# อ่านข้อมูลจาก Staging Zone
df = pd.read_csv(
    r"C:\Users\USER\Desktop\BI-DM Project\data\Staging_zone\staging_data.csv"
)

# เปลี่ยนชื่อ column ให้เข้าใจง่าย
df = df.rename(columns={
    "country_name": "country",
    "value": "co2"
})

# ลบข้อมูลซ้ำ
df = df.drop_duplicates()

# ลบแถวที่มี missing
df = df.dropna()

# แปลงชนิดข้อมูล
df["year"] = df["year"].astype(int)
df["co2"] = df["co2"].astype(float)

# บันทึก Cleansing Zone
df.to_csv(
    r"C:\Users\USER\Desktop\BI-DM Project\data\Cleansing_zone\clean_data.csv",
    index=False
)

print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13953 entries, 0 to 13952
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  13953 non-null  object 
 1   year     13953 non-null  int64  
 2   co2      13953 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 327.2+ KB
None


In [5]:
# Staging Zone
import pandas as pd

# อ่านไฟล์ raw จากเครื่อง
raw_csv = pd.read_csv(
    r"C:\Users\USER\Desktop\BI-DM Project\data\Raw_zone\raw_data.csv"
)

# เลือกเฉพาะ column ที่จำเป็น
cols = ["country_name", "year", "value"]
csv_df = raw_csv[cols]

# รวมข้อมูล (ตอนนี้มีไฟล์เดียว)
staging_df = pd.concat([csv_df], ignore_index=True)

# บันทึกไป Staging Zone
staging_df.to_csv(
    r"C:\Users\USER\Desktop\BI-DM Project\data\Staging_zone\staging_data.csv",
    index=False
)

print(staging_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13953 entries, 0 to 13952
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country_name  13953 non-null  object 
 1   year          13953 non-null  int64  
 2   value         13953 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 327.2+ KB
None


In [ ]:
#TEST
import pandas as pd

# อ่านข้อมูลดิบจาก Raw Zone
raw_csv = pd.read_csv(r"C:\Users\USER\Desktop\BI-DM Project\data\Raw_zone\raw_data.csv")


print(raw_csv.head())


  country_code country_name  year      value
0          ABW        Aruba  1960  11092.675
1          ABW        Aruba  1961  11576.719
2          ABW        Aruba  1962  12713.489
3          ABW        Aruba  1963  12178.107
4          ABW        Aruba  1964  11840.743
